<a href="https://colab.research.google.com/github/JixTheCat/sustainable_data_roo/blob/main/NGA_food_market_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [18]:
import os
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
import json

In [19]:
if not os.path.exists("sustainable_data_roo"):
  !git clone https://github.com/JixTheCat/sustainable_data_roo

%cd sustainable_data_roo

Cloning into 'sustainable_data_roo'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 57 (delta 20), reused 27 (delta 9), pack-reused 0
Receiving objects: 100% (57/57), 2.03 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/sustainable_data_roo/sustainable_data_roo/sustainable_data_roo


## Data Preprocessing

In [20]:
url = "https://raw.githubusercontent.com/JixTheCat/sustainable_data_roo/main/data/wfp_food_prices_nga.csv"

# Use pandas to read the CSV data from the URL
df = pd.read_csv(url)

<ipython-input-20-b5f77abb903d>:4: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [21]:
column_order = df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81691 entries, 0 to 81690
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       81691 non-null  object
 1   admin1     81691 non-null  object
 2   admin2     81691 non-null  object
 3   market     81691 non-null  object
 4   latitude   81691 non-null  object
 5   longitude  81691 non-null  object
 6   category   81691 non-null  object
 7   commodity  81691 non-null  object
 8   unit       81691 non-null  object
 9   priceflag  81691 non-null  object
 10  pricetype  81691 non-null  object
 11  currency   81691 non-null  object
 12  price      81691 non-null  object
 13  usdprice   81691 non-null  object
dtypes: object(14)
memory usage: 8.7+ MB


In [22]:
df = df[df["usdprice"] != 0]

df = df[df['commodity'].str.contains('Rice', case=False)]

df = df[df['unit'].str.contains('KG', case=False)]

df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

df['unit'] = df['unit'].str.replace('KG', '', case=False, regex=False).str.strip().replace('', '0', regex=False).astype(float)
df['usdprice'] = df['usdprice'].astype(float)

def update_columns(row):
    if not row['unit'] or row['unit'] == 0:
        row['unit'] = 100
        row['usdprice'] = row['usdprice'] * 100
    else:
        multiplier = 100 / row['unit']
        row['unit'] *= multiplier
        row['usdprice'] *= multiplier
    return row

df = df.apply(update_columns, axis=1)

# df = df.apply(update_columns, axis=1)

df = df.sort_values(by=['date'], ascending=[False])
df = df.groupby(['market']).first().reset_index()

# Reorder the columns to match the original order
df = df[column_order]

In [23]:
df.loc[df['unit'] == 100, 'unit'] = '100 KG'

df.to_csv('data/final_dataset_nga.csv', index=False)

## Preprocessing for routes

In [24]:
# Load the origin CSV file
origin_csv_path = 'data/final_dataset_nga.csv'

# Read the CSV file into a pandas DataFrame
origin_df = pd.read_csv(origin_csv_path)

# Display the first few rows of the DataFrame to understand its structure
origin_df.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,2023-01-15,Abia,Oboma Ngwa,Aba,5.147640,7.356720,cereals and tubers,Rice (imported),100 KG,aggregate,Wholesale,NGN,41960.0,182.147400
1,2023-06-15,Borno,Konduga,Abba Gamaram,11.858182,13.154892,cereals and tubers,Rice (local),100 KG,actual,Retail,NGN,1120.0,173.021429
2,2023-06-15,Yobe,Karasuwa,Bade (Gashua),12.868665,11.039624,cereals and tubers,Rice (local),100 KG,actual,Retail,NGN,994.0,153.550000
3,2023-06-15,Borno,Konduga,Baga Road,11.867950,13.123314,cereals and tubers,Rice (local),100 KG,actual,Retail,NGN,1127.0,174.100000
4,2023-01-15,Borno,Biu,Biu,10.611100,12.195000,cereals and tubers,Rice (local),100 KG,aggregate,Retail,NGN,413.6,89.770000


In [25]:
# Extract the relevant columns for finding the closest markets
coordinates = origin_df[['latitude', 'longitude']].values

# Using NearestNeighbors to find the closest points
# We set n_neighbors=6 because the results will include the point itself as the first entry
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(coordinates)
distances, indices = nbrs.kneighbors(coordinates)

# Create a new DataFrame for the destination CSV
# Initialize it with columns from the origin CSV and add 'market_dest'
destination_columns = origin_df.columns.tolist() + ['market_dest']
# Re-create the destination DataFrame without including the market itself as a destination
destination_df = pd.DataFrame(columns=destination_columns)

# Populate the new DataFrame without including the origin market as one of its own destinations
for i, market in origin_df.iterrows():
    # Get the indices of the five closest markets (excluding the market itself which is at index 0)
    closest_indices = indices[i][1:]  # Skip the first one as it is the market itself
    for idx in closest_indices:
        # Copy the row of the closest market
        row_data = origin_df.iloc[idx].copy()
        # Set the market_dest to the name of the target market
        row_data['market_dest'] = market['market']
        # Append the row to the destination DataFrame
        destination_df = destination_df.append(row_data, ignore_index=True)

# Save the corrected DataFrame to a CSV file
destination_csv_path_corrected = 'data/destination_data_corrected.csv'
destination_df.to_csv(destination_csv_path_corrected, index=False)

<ipython-input-25-e08e292d180c>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destination_df = destination_df.append(row_data, ignore_index=True)
<ipython-input-25-e08e292d180c>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destination_df = destination_df.append(row_data, ignore_index=True)
<ipython-input-25-e08e292d180c>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destination_df = destination_df.append(row_data, ignore_index=True)
<ipython-input-25-e08e292d180c>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destination_df = destination_df.append(row_data, ignore_index=True)
<ipython-input-25-e08e292d180c>:25: FutureWa

In [26]:
# Load the CSV files
origin_data_path = 'data/final_dataset_nga.csv'
destination_data_path = 'data/destination_data_corrected.csv'

# Read the CSV files into pandas DataFrames
origin_data_df = pd.read_csv(origin_data_path)
destination_data_df = pd.read_csv(destination_data_path)

# Display the first few rows of each DataFrame to understand their structure
origin_data_head = origin_data_df.head()
destination_data_head = destination_data_df.head()

(origin_data_head, destination_data_head)

(         date admin1      admin2         market   latitude  longitude  \
 0  2023-01-15   Abia  Oboma Ngwa            Aba   5.147640   7.356720   
 1  2023-06-15  Borno     Konduga   Abba Gamaram  11.858182  13.154892   
 2  2023-06-15   Yobe    Karasuwa  Bade (Gashua)  12.868665  11.039624   
 3  2023-06-15  Borno     Konduga      Baga Road  11.867950  13.123314   
 4  2023-01-15  Borno         Biu            Biu  10.611100  12.195000   
 
              category        commodity    unit  priceflag  pricetype currency  \
 0  cereals and tubers  Rice (imported)  100 KG  aggregate  Wholesale      NGN   
 1  cereals and tubers     Rice (local)  100 KG     actual     Retail      NGN   
 2  cereals and tubers     Rice (local)  100 KG     actual     Retail      NGN   
 3  cereals and tubers     Rice (local)  100 KG     actual     Retail      NGN   
 4  cereals and tubers     Rice (local)  100 KG  aggregate     Retail      NGN   
 
      price    usdprice  
 0  41960.0  182.147400  
 1   112

In [27]:
# Function to convert DataFrame to GeoJSON
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon], row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [28]:
# Convert the DataFrame to GeoJSON
origin_geojson = df_to_geojson(origin_data_df, origin_data_df.columns)
destination_geojson_corrected = df_to_geojson(destination_data_df, destination_data_df.columns)

# Save the GeoJSON to a file
origin_geojson_path = 'data/final_dataset_nga.geojson'
destination_geojson_path_corrected = 'data/destination_data_corrected.geojson'
with open(origin_geojson_path, 'w') as f:
    json.dump(origin_geojson, f)

with open(destination_geojson_path_corrected, 'w') as f:
    json.dump(destination_geojson_corrected, f)